#Transfer learning a partir do modelo **VGG16**

##Adaptando-o a um problema de classificação de doenças em plantas.

###Esse exemplo utiliza ImageDataGenerator para carregar e aumentar os dados, e mostra como congelar as camadas iniciais do VGG16 e adicionar novas camadas para a tarefa específica.

Você pode adaptar esse exemplo para ResNet ou outros modelos pré-treinados seguindo uma lógica semelhante.


# Importar pacotes necessários

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16 # ou use ResNet50, ResNet101 conforme desejado
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Parâmetros do treinamento

In [ ]:
img_height, img_width = 224, 224 # Tamanho esperado pela VGG16
batch_size = 32
epochs = 20
num_classes = 3 # Exemplo: número de categorias de doenças em plantas

# Diretórios com os dados ()

In [ ]:
train_data_dir = 'caminho/para/train'
validation_data_dir = 'caminho/para/validation'

# Pré-processamento e aumento de dados

In [ ]:
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=30,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size=(img_height, img_width),
batch_size=batch_size,
class_mode='categorical'
)
validation_generator = val_datagen.flow_from_directory(
validation_data_dir,
target_size=(img_height, img_width),
batch_size=batch_size,
class_mode='categorical'
)

# Carregar o modelo VGG16 pré-treinado sem as camadas de classificação (top)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Congelar as camadas do modelo base para não alterar os pesos pré-treinados

In [ ]:
for layer in base_model.layers:
layer.trainable = False

# Adicionar novas camadas para a nossa tarefa específica

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x) # Reduz a dimensão e extrai features globais
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Modelo final para fine-tuning

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar o modelo

In [ ]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Visualizar o resumo do modelo montado

In [ ]:
model.summary()

# Treinar o modelo

In [ ]:
history = model.fit(
train_generator,
steps_per_epoch=train_generator.samples // batch_size,
epochs=epochs,
validation_data=validation_generator,
validation_steps=validation_generator.samples // batch_size
)